# All Function Definitions

In [2]:
def convert_to_TA(array,freq): #This function converts thermal temp to antenna temp using the standard TA=T*x/(e^x-1) formula
    for x in range(len(array)):
        if not array[x]<=0 and not array[x]==np.inf:
            IGL=(6.62607015*(10**(-34))*freq*(10**6))/(1.380649*(10**(-23))*array[x])
            array[x]=(IGL/(np.exp(IGL)-1))*array[x]
    return array

In [3]:
def convert_to_TT(temp_array,freq): #This function does the opposite, and converts to thermal temp
    
    for i in range(len(temp_array)):
        TA=temp_array[i]
        convert_to_thermal_temp = lambda TT : TA-((6.62607015*(10**(-34))*freq*10**6)/(1.380649*(10**(-23))*TT)/
                       (np.exp((6.62607015*(10**(-34))*freq*10**6)/(1.380649*(10**(-23))*TT))-1))*TT 
        TT_guess = TA
        TT = fsolve(convert_to_thermal_temp, TT_guess)
        temp_array[i]=TT[0]
    return temp_array

In [4]:
def MAP_Rotation_to_G(array,coords):
    #A rotator function which rotates a map to G. Does a fair amount of smoothing-- avoid if possible
    npix=int(np.sqrt(len(array)/12)) #12*Nside^2=total number of pixels
    theta1,phi1=hp.pix2ang(npix,np.arange(len(array)))
    if coords=="E":
        r = hp.rotator.Rotator(coord=["G","E"])
        theta,phi=(r(theta1,phi1))
    if coords=="C":
        r = hp.rotator.Rotator(coord=["G","C"])
        theta,phi=(r(theta1,phi1))
    array = hp.get_interp_val(array, theta, phi)
    return array

In [1]:
def TG_calculation(files):
    #This function calculates T_g from each method and uses ARCADE's uncertainty as the uncertainty
    T_g_array=[]
    T_g_unc=[]
    for file in tqdm(files):
        c_ARCADE,c_unc_ARCADE,T_g_ARCADE,T_g_unc_ARCADE,freq=cscb_Temp_fit_ARCADE(file,"North",False)
        c_hol,c_unc_hol,T_g_hol,T_g_unc_hol,freq=cscb_Temp_fit_hol(file,"North",False)
        c_hol_mean,c_unc_hol_mean,T_g_hol_mean,T_g_unc_hol_mean,freq=cscb_Temp_fit_hol_mean(file,"North",False)
        T_g=np.mean([T_g_ARCADE,T_g_hol,T_g_hol_mean])
        T_g_array.append([T_g])
        T_g_unc.append([T_g_unc_ARCADE])
        c_ARCADE,c_unc_ARCADE,T_g_ARCADE,T_g_unc_ARCADE,freq=cscb_Temp_fit_ARCADE(file,"South",False)
        c_hol,c_unc_hol,T_g_hol,T_g_unc_hol,freq=cscb_Temp_fit_hol(file,"South",False)
        c_hol_mean,c_unc_hol_mean,T_g_hol_mean,T_g_unc_hol_mean,freq=cscb_Temp_fit_hol_mean(file,"South",False)
        T_g=np.mean([T_g_ARCADE,T_g_hol,T_g_hol_mean])
        index=files.index(file)
        T_g_array[index].append(T_g)
        T_g_unc[index].append(T_g_unc_ARCADE)
    return T_g_array,T_g_unc

In [2]:
def DynestyFit(dim,loglike,ptlist,graph):
    #Performs a dynesty fit
    #dim=dimension of the problem
    #loklike is the likelihood function
    #ptlist is a list of length dim, which goes as [max,min] for each parameter involved
    #graph is a boolean to determine whether the parameters' are graphed
    ndim=dim

    def ptform(u):
        x = np.array(u)
        for i in range(dim):
            rang=(ptlist[i][0]-ptlist[i][1])
            x[i]=rang*u[i]+ptlist[i][1]
        return x
    
    # "Static" nested sampling.
    sampler = dynesty.NestedSampler(loglike, ptform, ndim)
    sampler.run_nested()
    sresults = sampler.results
    dsampler = dynesty.DynamicNestedSampler(loglike, ptform, ndim,nlive=500)
    dsampler.run_nested()
    dresults = dsampler.results
    results = dyfunc.merge_runs([sresults, dresults])

    if graph==True:
        from dynesty import plotting as dyplot
        cfig, caxes = dyplot.cornerplot(results,show_titles=True)

    samples=results.samples
    weights=np.exp(results["logwt"]-results["logz"][-1])
    quantiles=[dyfunc.quantile(samps,[0.159,0.5,0.841],weights=weights) for samps in samples.T]
    const=[]
    const_err=[]
    for x in quantiles:
        const.append(x[1])
        const_err.append((abs(x[1]-x[0])+abs(x[1]-x[2]))/2)
    return const,const_err

In [1]:
def cscb_Temp_fit_ARCADE(file,region,graph):
    
    #We first convert the array to a list to go faster
    if file[1]==1420:#1420 MHz is the only one on this list (so far) that is in mK
        File_as_List=(hp.read_map(file[0])/1000).tolist()
    else:
        File_as_List=(hp.read_map(file[0])).tolist()
    
    #If we have the uncertainty map, we read it and convert it to a list
    if type(file[4])==str:
        Unc_as_List=(hp.read_map(file[4])).tolist()
    elif type(file[4])==list:
        Unc_as_List=(np.sqrt((0.01*file[4][1]*np.array(File_as_List))**2)).tolist()
    
    #If we start in thermo-temp, we make sure to converto to antenna temp:
    if file[3]==True:
        convert_to_TA(File_as_List,file[1])
        
    #Next, we define our theta coords to bin each value by csc|b|, making sure to go to galactic coords
    npix=int(np.sqrt(len(File_as_List)/12)) #12*Nside^2=total number of pixels
    theta1,phi1=hp.pix2ang(npix,np.arange(len(File_as_List)))
    if file[2]=="E":
        r = hp.rotator.Rotator(coord=['E','G'])
        theta=np.degrees(r(theta1,phi1)[0])
    if file[2]=="C":
        r = hp.rotator.Rotator(coord=['C','G'])
        theta=np.degrees(r(theta1,phi1)[0])
    if file[2]=="G":
        theta=np.degrees(theta1)
    
    cscb_Temp_values=[[],[],[],[],[],[],[],[],[],[]] #each entry represents an 0.5 increase from 1.25 (1.25,1.75,2.25...)
    unc_Temp_values=[[],[],[],[],[],[],[],[],[],[]]
    csc_list=[[],[],[],[],[],[],[],[],[],[]]
    #Now, we bin each actual temp (AKA, not inf, negative, or NaN) and its corresponding theta (b or latitude) by csc|b|
    for x in (range(int(len(File_as_List)))):
        if File_as_List[x]>0 and math.isnan(File_as_List[x])==False and Unc_as_List[x]>=0: #ignore hp.UNSEEN and 0 temp pixels
            
            #(2326 MHz has an uncertain absolute temp, we correct that)
            if file[1]==2326:
                File_as_List[x]+=2.505+0.325*(2326/2000)**(-2.75)
            
            #We then may choose to look at the Northern hemisphere, the Southern one, or both.
            if region=="North":
                condition=90-theta[x]>10
            elif region=="South":
                condition=90-theta[x]<-10
            elif region=="All":
                condition=abs(90-theta[x])>10
                
            if condition: # We ignore the galactic center: 10>|b|
                range_csc=0 #dummy var to index cscb_Temp_values
                csc=1/np.sin(abs(np.radians(90-theta[x])))
                while range_csc<=9:
                    if range_csc/2+1<=csc<range_csc/2+1.5: #append all values within 0.25 csc |b|
                        cscb_Temp_values[range_csc].append(File_as_List[x])
                        unc_Temp_values[range_csc].append(Unc_as_List[x])
                        csc_list[range_csc].append(csc)
                        range_csc=range_csc+10
                    else:
                        range_csc=range_csc+1 #if the entry is not within 1<x<1.5, increase to 1.5<x<2.0 (for 1.75), etc.

    Mean_cscb_Temp=[]
    Mean_csc_list=[]
    cal_Unc_array=[]
    cscb_Temp_sigma=[]
    ARCADE_arr=[]
    num_data_points=0
    for i in range(10):
        if not cscb_Temp_values[i]==[]: #if there are any values within the range of csc we defined above:
            Mean_cscb_Temp.append(np.mean(cscb_Temp_values[i])) # we mean the temperature and create a scatter plot
            Mean_csc_list.append(np.mean(csc_list[i])) # we also mean csc
            #ARCADE_arr.append(np.sqrt((file[4][0]**2)/len(cscb_Temp_values[i]))) #for replicating the graph in the ARCADE paper
            
            if file[3]==False:
                if type(file[4])==list:
                    cscb_stat_sigma.append(file[4][0]/np.sqrt(len(cscb_Temp_values[i]))) #we define stat unc. to use for inflation
                    if file[1]==45 and region=="South":
                        cscb_stat_sigma=(np.array(cscb_Temp_sigma)*2300/300).tolist()
                else:
                    cscb_Temp_sigma.append(np.sqrt(np.sum(np.array(unc_Temp_values[i])**2/len(unc_Temp_values[i])**2)))
            else:
                #We convert uncertainty to antenna temp (though to third order they are the same)
                IGL=(6.62607015*(10**(-34))*file[1]*(10**6))/(1.380649*(10**(-23))*np.array(cscb_Temp_values[i]))
                cscb_Temp_sigma.append(np.sqrt(np.sum((file[4][0]*(IGL**2)*np.exp(IGL)/((np.exp(IGL)-1)**2))**2)/(len(cscb_Temp_values[i]))**2))

            cal_Unc_array.append(np.sqrt(np.sum(np.array(unc_Temp_values[i])**2/len(unc_Temp_values[i])**2))) #we define cal uncertainty
            num_data_points+=1 
    Mean_cscb_Temp = np.array(Mean_cscb_Temp)
    
    #first we do our fit with only statistical uncertainty
    ChiSquared = lambda T: np.sum(((T[0]+T[1]*np.array(Mean_csc_list)-Mean_cscb_Temp)/(cscb_stat_sigma))**2)
    bnds = ((0, np.inf),(0, np.inf))
    results=minimize(ChiSquared,x0=(np.mean(Mean_cscb_Temp),np.mean(Mean_cscb_Temp)),bounds=bnds)
    if results.fun/(num_data_points-2)>=1: #if the results are >1, we inflate the uncertainties until chi^2/dof=1
        cscb_stat_sigma=np.array(cscb_stat_sigma)*np.sqrt(results.fun/(num_data_points-2))

    if type(file[4])==list:
        sgma=np.sqrt(np.array(cscb_stat_sigma)**2+np.array(cal_Unc_array)**2) #we then add calibration error and refit
    else:
        sgma=cscb_Temp_sigma #the LWA maps do not distinguish between types of uncertainties
        
    def func(x,a,b):
        return a*x+b
    p,cov=curve_fit(func, Mean_csc_list, Mean_cscb_Temp,sigma=sgma,absolute_sigma=True)
    T_g=p[0]
    c=p[1]
    T_g_unc,c_unc=np.sqrt(np.diag(cov))
    if type(file[4])==list:
        c_unc=np.sqrt(c_unc**2)
        #Plotting uncertainty:
    """
    #For if we want to produce the graphs in the ARCADE paper
    if file[1]==408 or file[1]==3150:
        if file[1]==408:
            plt.errorbar(x=Mean_csc_list,y=Mean_cscb_Temp,yerr=np.array(ARCADE_arr)*20,fmt="none",color="maroon",capsize=2)
            plt.scatter(Mean_csc_list,Mean_cscb_Temp,color="red")
            plt.plot(Mean_csc_list,T_g*np.array(Mean_csc_list)+c)
            plt.title("csc|b| vs Temp at %s MHz" % file[1])
            plt.xlabel("csc|b|")
            plt.ylabel("Antenna Temp (K)")
            plt.grid()
            plt.minorticks_on()
            #plt.savefig("T_g_408_MHz_ARCADE_Replica_in_Paper",dpi=200)
        if file[1]==3150:
            plt.errorbar(x=Mean_csc_list,y=Mean_cscb_Temp,yerr=np.array(ARCADE_arr)*5,fmt="none",color="maroon",capsize=2)
            plt.scatter(Mean_csc_list,Mean_cscb_Temp,color="red")
            plt.plot(Mean_csc_list,T_g*np.array(Mean_csc_list)+c)
            plt.title("csc|b| vs Temp at %s MHz" % file[1])
            plt.xlabel("csc|b|")
            plt.ylabel("Antenna Temp (K)")
            plt.grid()
            plt.minorticks_on()
            #plt.savefig("T_g_3150_MHz_ARCADE_Replica_in_Paper",dpi=200)
        plt.show()
    """
    if graph==True:
        #Plotting uncertainty:
        plt.errorbar(x=Mean_csc_list,y=Mean_cscb_Temp,yerr=sgma,fmt="none",color="maroon",capsize=2)
        #Plotting the scatter:
        plt.scatter(Mean_csc_list,Mean_cscb_Temp,color="red")
        #Fitting the scatter and plotting:
        plt.plot(Mean_csc_list,T_g*np.array(Mean_csc_list)+c)
        plt.title("csc|b| vs Temp at %s MHz" % file[1])
        plt.xlabel("csc|b|")
        plt.ylabel("Antenna Temp (K)")
        plt.grid()
        plt.show()
        print("T_extra at %sMHz is %.5f plus/minus %.5f" % (file[1], c,c_unc))
        print("T_g at %sMHz is %.5f plus/minus %.5f" % (file[1],T_g,T_g_unc))
    return c,c_unc,T_g,T_g_unc,file[1]


In [10]:
def cscb_Temp_fit_hol(file,region,graph):
    
    #We first convert the array to a list to go faster
    if file[1]==1420:#1420 MHz is the only one on this list (so far) that is in mK
        File_as_List=(hp.read_map(file[0])/1000).tolist()
    else:
        File_as_List=(hp.read_map(file[0])).tolist()

    #If we have an uncertainty map, we read it and make it a list
    if type(file[4])==str:
        Unc_as_List=(hp.read_map(file[4])).tolist()
    elif type(file[4])==list:
        Unc_as_List=((np.sqrt((0.01*file[4][1]*np.array(File_as_List))**2))+0.000001).tolist()

    #If we start in thermo-temp, we make sure to converto to antenna temp:
    
    if file[3]==True:
        convert_to_TA(File_as_List,file[1])
        
    #Next, we define our theta coords to bin each value by csc|b|, making sure to go to galactic coords
    npix=int(np.sqrt(len(File_as_List)/12)) #12*NPix^2=total number of pixels
    theta1,phi1=hp.pix2ang(npix,np.arange(len(File_as_List)))
    if file[2]=="E":
        r = hp.rotator.Rotator(coord=['E','G'])
        theta=np.degrees(r(theta1,phi1)[0])
        phi=np.degrees(r(theta1,phi1)[1])+180 #phi rotates oddly.
    if file[2]=="C":
        r = hp.rotator.Rotator(coord=['C','G'])
        theta=np.degrees(r(theta1,phi1)[0])
        phi=np.degrees(r(theta1,phi1)[1])+180
    if file[2]=="G":
        theta=np.degrees(theta1)
        phi=np.degrees(phi1)
    
    #Now we calculate csc|b| for each pixel and add it and its temperature to a list
    Tempvalues=[]
    cscvalues=[]
    uncvals=[]
    for x in (range(int(len(File_as_List)))):
        if File_as_List[x]>0 and math.isnan(File_as_List[x])==False and Unc_as_List[x]>0: #ignore hp.UNSEEN and 0 pixels
            if file[1]==2326:
                File_as_List[x]+=2.505+0.325*(2326/2000)**(-2.75)
            #We may look at the northern hemisphere, southern, or both
            if region=="North" or region=="All":
                if 90-theta[x]>10:
                    Tempvalues.append(File_as_List[x])
                    csc=1/np.sin(np.radians(abs(90-theta[x])))
                    cscvalues.append(csc)
                    uncvals.append(Unc_as_List[x])
            if region=="South" or region=="All":
                if 90-theta[x]<-10:
                    Tempvalues.append(File_as_List[x])
                    csc=1/np.sin(np.radians(abs(90-theta[x])))
                    cscvalues.append(csc)
                    uncvals.append(Unc_as_List[x])

    #Inflating error bars:
    Tempvalues = np.array(Tempvalues)
    if type(file[4])==list:
        cscb_stat_sigma=0*np.arange(len(Tempvalues))+file[4][0] #n=1 here, so no need to divide by sqrt(n)
        if file[3]==True:
            #Change to T_A
            IGL=(6.62607015*(10**(-34))*file[1]*(10**6))/(1.380649*(10**(-23))*np.array(Tempvalues))
            cscb_stat_sigma=file[4][0]*(IGL**2)*np.exp(IGL)/((np.exp(IGL)-1)**2)
    if file[3]==False and type(file[4])==list:
        if file[1]==45 and region=="South":
            cscb_Temp_sigma=np.array(cscb_Temp_sigma)+2000
        ChiSquared = lambda T: np.sum(((T[0]+T[1]*np.array(cscvalues)-Tempvalues)/(cscb_stat_sigma))**2)
        bnds = ((0, np.inf),(0, np.inf))
        results=minimize(ChiSquared,x0=(np.mean(Tempvalues),np.mean(Tempvalues)),bounds=bnds)
        
        if results.fun/(len(Tempvalues)-2)>=1:
            cscb_stat_sigma=np.array(cscb_stat_sigma)*np.sqrt(results.fun/(len(Tempvalues)-2))
    
    if type(file[4])==list:
        sgma=np.sqrt(np.array(cscb_stat_sigma)**2+np.array(uncvals)**2)
    else:
        sgma=uncvals
        
    def func(x,a,b):
        return a*x+b
    p,cov=curve_fit(func, cscvalues, Tempvalues,sigma=sgma,absolute_sigma=True)
    T_g=p[0]
    c=p[1]
    T_g_unc,c_unc=np.sqrt(np.diag(cov))
    if type(file[4])==list:
        c_unc=np.sqrt(c_unc**2+file[4][2]**2)
    if graph==True:
        if type(file[4])==str:
            plt.fill_between(cscvalues, Tempvalues-uncvals, Tempvalues+uncvals,facecolor="pink")
        plt.scatter(cscvalues,Tempvalues,s=10,color="red")
        plt.plot(cscvalues,T_g*np.array(cscvalues)+c)
        plt.title("csc|b| vs Temp at %s MHz" % file[1])
        plt.xlabel("csc|b|")
        plt.ylabel("Antenna Temp (K)")
        plt.grid()
        plt.show()
        print("T_extra at %sMHz is %.5f plus/minus %.5f" % (file[1], c,c_unc))
        print("T_g at %sMHz is %.5f plus/minus %.5f"% (file[1],T_g,T_g_unc))
    return c,c_unc,T_g,T_g_unc,file[1]


In [7]:
def cscb_Temp_fit_hol_mean(file,region,graph):

    #We first convert the array to a list to go faster
    if file[1]==1420:#1420 MHz is the only one on this list (so far) that is in mK
        File_as_List=(hp.read_map(file[0])/1000).tolist()
    else:
        File_as_List=(hp.read_map(file[0])).tolist()
        
    #If the file has an uncertainty map, we read it, and turn it into a list
    if type(file[4])==str:
        Unc_as_List=(hp.read_map(file[4])).tolist()
    elif type(file[4])==list:
        Unc_as_List=(np.sqrt((0.01*file[4][1]*np.array(File_as_List))**2)).tolist()
    
    #If we start in thermo-temp, we make sure to converto to antenna temp:
    if file[3]==True:
        convert_to_TA(File_as_List,file[1])
        TA_conv_arr=[]
        for x in range(len(File_as_List)):
            if File_as_List[x]>0:
                TA_conv_arr.append(File_as_List[x]/hp.read_map(file[0])[x])
        TA_conv_factor=np.mean(TA_conv_arr)
        
    #Next, we define our theta coords to bin each value by csc|b|, making sure to go to galactic
    npix=int(np.sqrt(len(File_as_List)/12)) #12*NPix^2=total number of pixels
    theta1,phi1=hp.pix2ang(npix,np.arange(len(File_as_List)))
    if file[2]=="E":
        r = hp.rotator.Rotator(coord=['E','G'])
        theta=np.degrees(r(theta1,phi1)[0])
        phi=np.degrees(r(theta1,phi1)[1])+180
    if file[2]=="C":
        r = hp.rotator.Rotator(coord=['C','G'])
        theta=np.degrees(r(theta1,phi1)[0])
        phi=np.degrees(r(theta1,phi1)[1])+180
    if file[2]=="G":
        theta=np.degrees(theta1)
        phi=np.degrees(phi1)
    
    Tempvalues=[]
    cscvalues=[]
    uncvals=[]
    Correction_to_45Mhz_RMS=0
    for x in (range(int(len(File_as_List)))): #We start off by doing what we did in the previous method
        if File_as_List[x]>0 and math.isnan(File_as_List[x])==False and Unc_as_List[x]>=0: #ignore hp.UNSEEN and 0 pixels   
            if file[1]==2326:
                File_as_List[x]+=2.505+0.325*(2326/2000)**(-2.75)
            if region=="North" or region=="All":
                if 70>90-theta[x]>10:
                    Tempvalues.append(File_as_List[x])
                    csc=1/np.sin(np.radians(abs(90-theta[x])))
                    cscvalues.append(csc) 
                    uncvals.append(Unc_as_List[x])
            if region=="South" or region=="All":
                if -70<90-theta[x]<-10:
                    Tempvalues.append(File_as_List[x])
                    csc=1/np.sin(np.radians(abs(90-theta[x])))
                    cscvalues.append(csc)
                    if file[1]==45 and type(file[4])==list and Correction_to_45Mhz_RMS==0: #45 MHz has different north and south rms unc
                        Unc_as_List=np.sqrt(np.array(Unc_as_List)**2-300**2+2300**2).tolist()
                        Correction_to_45Mhz_RMS=Correction_to_45Mhz_RMS+1
                    uncvals.append(Unc_as_List[x])
    
    Mean_Temp_List=[]
    cal_Unc_array=[]
    cscb_stat_sigma=[]
    
    for x in (range(len(cscvalues))):
        cscvalues[x]=round(cscvalues[x], 2) #Now we round each csc value to the hundredths
    cscset=list(set(cscvalues)) #we define a set of all of the UNIQUE csc values
    cscvalues=np.array(cscvalues)
    for x in (cscset):
        DummyTempList=[]
        DummyUncList=[]
        indices=np.where(cscvalues==x)[0] #we look at what indices the csc values are equal...
        for i in indices:
            DummyTempList.append(Tempvalues[i])
            DummyUncList.append(uncvals[i])
        Mean_Temp_List.append(np.mean(DummyTempList))
        if file[3]==False:
            if type(file[4])==list:
                cscb_stat_sigma.append(file[4][0]/np.sqrt(len(indices)))
                if file[1]==45 and region=="South":
                    cscb_stat_sigma=(np.array(cscb_stat_sigma)*2300/300).tolist()
        else:
            IGL=(6.62607015*(10**(-34))*file[1]*(10**6))/(1.380649*(10**(-23))*np.array(DummyTempList))
            cscb_stat_sigma.append(np.sqrt(np.sum((file[4][0]*(IGL**2)*np.exp(IGL)/((np.exp(IGL)-1)**2))**2)/(len(DummyTempList))**2))
        cal_Unc_array.append(np.sqrt(np.sum(np.array(DummyUncList)**2/len(indices)**2)))

    #Due to the fill-between function reading cscvalues in order, and yet, they are all jumbled, we have to order
    #the csc values and their temps/uncertainties:
    Sortedvals=[]
    for x in range(len(cscset)):
        Sortedvals.append([cscset[x],Mean_Temp_List[x],cal_Unc_array[x]])
    Sortedvals=np.array(sorted(Sortedvals)).transpose()
    cscset=Sortedvals[0]
    Mean_Temp_List=Sortedvals[1]
    cal_Unc_array=Sortedvals[2]

    #inflate error bars:
    if file[3]==False and type(file[4])==list:
        if file[1]==45 and region=="South":
            cscb_stat_sigma=np.array(cscb_stat_sigma)+2000
        ChiSquared = lambda T: np.sum(((T[0]+T[1]*np.array(cscset)-Mean_Temp_List)/(cscb_stat_sigma))**2)
        bnds = ((0, np.inf),(0, np.inf))
        results=minimize(ChiSquared,x0=(np.mean(Mean_Temp_List),np.mean(Mean_Temp_List)),bounds=bnds)
        if results.fun/(len(cscset)-2)>=1:
            cscb_stat_sigma=np.array(cscb_stat_sigma)*np.sqrt(results.fun/(len(cscset)-2))
    
    if type(file[4])==list:
        sgma=np.sqrt(np.array(cscb_stat_sigma)**2+np.array(cal_Unc_array)**2)
    else:
        sgma=cal_Unc_array
    def func(x,a,b):
        return a*x+b
    p,cov=curve_fit(func, cscset, Mean_Temp_List,sigma=sgma,absolute_sigma=True)
    T_g=p[0]
    c=p[1]
    T_g_unc,c_unc=np.sqrt(np.diag(cov))
    if type(file[4])==list:
        c_unc=np.sqrt(c_unc**2+file[4][2]**2)
    
    if graph==True:
        plt.scatter(cscset,Mean_Temp_List,s=3,color="red")
        plt.plot(cscset,T_g*np.array(cscset)+c)
        plt.title("csc|b| vs Temp at %s MHz" % file[1])
        plt.xlabel("csc|b|")
        plt.ylabel("Antenna Temp (K)")
        plt.grid()
        plt.show()
        print("T_extra at %sMHz is %.5f plus/minus %.5f" % (file[1], c,c_unc))
        print("T_g at %sMHz is %.5f plus/minus %.5f"% (file[1],T_g,T_g_unc))
    return c,c_unc,T_g,T_g_unc,file[1]

In [3]:
def ChiSquaredMinimizer(file,T_g_array,T_g_unc):
    if file[1]==1420:#1420 MHz is the only one on this list (so far) that is in mK
        Map_as_array=hp.ud_grade((hp.read_map(file[0])/1000),16)
    else:
        Map_as_array=hp.ud_grade((hp.read_map(file[0])),16)
    
    nside=int(np.sqrt(len(Map_as_array)/12))
    FreeFree_Map=(hp.ud_grade(hp.read_map("lambda_fds_dust_94GHz.fits"),nside)).tolist()
    #Synch_Map=hp.ud_grade((hp.read_map("wmap_K_mem_synch_9yr_v5.fits")),nside).tolist()
    Synch_Map=hp.ud_grade(hp.ud_grade(hp.read_map("haslam408_dsds_Remazeilles2014.fits"),128)
                           -np.array(Rescaled408map)
                          ,nside).tolist()
    #We convert all maps to TA-- no worries about uncertainty values since they are all the same to order 3
    if file[3]==True:
        convert_to_TA(Map_as_array,file[1])    
    
    theta,phi=np.degrees(hp.pix2ang(nside,np.arange(len(Map_as_array))))
    
    #Here we must rotate as we are subtracting maps
    if file[2]=="E":
        Map_as_array=MAP_Rotation_to_G(Map_as_array,"E")
    if file[2]=="C":
        Map_as_array=MAP_Rotation_to_G(Map_as_array,"C")
    Map_as_array=Map_as_array.tolist()
    
    Map_cut=[]
    Freefree_map_cut=[]
    Synch_map_cut=[]
    
    #We rotate the uncertainty maps as well for the LWAs
    if type(file[4])==str:
        sigma=[]
        sigmaMap=hp.read_map(file[4])
        if file[2]=="E":
            sigmaMap=MAP_Rotation_to_G(sigmaMap,"E")
        if file[2]=="C":
            sigmaMap=MAP_Rotation_to_G(sigmaMap,"C")
        sigmaMap=sigmaMap.tolist()
    
    #We only use pixels whose temp is >0, outside of |b|<10, and where the synchrotron map behaves properly
    for x in range(len(Map_as_array)):
        if Map_as_array[x]>0 and math.isnan(Map_as_array[x])==False and abs(90-theta[x])>10 and Synch_Map[x]>=0:
            if file[1]==2326:
                Map_as_array[x]+=2.505+0.325*(2326/2000)**(-2.75)
            if type(file[4])==list:            
                Map_cut.append(Map_as_array[x])
                Freefree_map_cut.append(FreeFree_Map[x])
                Synch_map_cut.append(Synch_Map[x])
            if type(file[4])==str and sigmaMap[x]>0:
                sigma.append(sigmaMap[x])
                Map_cut.append(Map_as_array[x])
                Freefree_map_cut.append(FreeFree_Map[x])
                Synch_map_cut.append(Synch_Map[x])
    if type(file[4])==list:
        #we include in our uncertainty RMS and calibration
        sigma=(np.sqrt(file[4][0]**2+(0.01*file[4][1]*np.array(Map_cut))**2)).tolist()

    #fitting
    def loglike(alpha):
        return -0.5 * np.sum(((np.array(Map_cut)-alpha[0]-alpha[1]*np.array(Freefree_map_cut)
                             -alpha[2]*np.array(Synch_map_cut))/(np.array(sigma)))**2)
        
    alpha,alpha_err=DynestyFit(3,loglike,[[np.max(Map_cut),-np.max(Map_cut)],[2*np.max(Map_cut),0],[2*np.max(Map_cut),0]],False)
    
    FreefreedumNorth=[]
    SynchdumNorth=[]
    FreefreedumSouth=[]
    SynchdumSouth=[]
    theta_N=[]
    theta_S=[]

    for x in range(len(Map_as_array)):
        if 90-theta[x]>75:
            FreefreedumNorth.append(FreeFree_Map[x])
            SynchdumNorth.append(Synch_Map[x])
            theta_N.append(90-theta[x])
        if 90-theta[x]<-75:
            FreefreedumSouth.append(FreeFree_Map[x])
            SynchdumSouth.append(Synch_Map[x])
            theta_S.append(90-theta[x])
    
    #For north and south we calculate from our formula the uncertainty and T_exc value
    theta_N=np.array(theta_N)
    theta_S=np.array(theta_S)
    
    T_g_N=T_g_array[files.index(file)][0]*1/np.sin(np.radians(abs(theta_N)))
    
    T_ff_N=np.array(FreefreedumNorth)*alpha[1]
    T_ff_N_err=np.sqrt((np.mean(FreefreedumNorth)*alpha_err[1])**2+(alpha[1]*np.std(FreefreedumNorth))**2)
    T_s_N=np.array(SynchdumNorth)*alpha[2]
    T_s_N_err=np.sqrt((np.mean(SynchdumNorth)*alpha_err[2])**2+(alpha[2]*np.std(SynchdumNorth))**2)
    
    T_exc_N=np.mean(alpha[0]+(T_ff_N+T_s_N-T_g_N))
    T_exc_N_err=(np.sqrt(alpha_err[0]**2+(T_ff_N_err**2+T_s_N_err**2+T_g_unc[files.index(file)][0]**2)))

    T_g_S=T_g_array[files.index(file)][1]*1/np.sin(np.radians(abs(theta_S)))
    
    T_ff_S=np.array(FreefreedumSouth)*alpha[1]
    T_ff_S_err=np.sqrt((np.mean(FreefreedumSouth)*alpha_err[1])**2+(alpha[1]*np.std(FreefreedumSouth))**2)
    T_s_S=np.array(SynchdumSouth)*alpha[2]
    T_s_S_err=np.sqrt((np.mean(SynchdumSouth)*alpha_err[2])**2+(alpha[2]*np.std(SynchdumSouth))**2)
    T_exc_S=np.mean(alpha[0]+(T_ff_S+T_s_S-T_g_S))
    T_exc_S_err=(np.sqrt(alpha_err[0]**2+(T_ff_S_err**2+T_s_S_err**2+T_g_unc[files.index(file)][1]**2)))

    #We make sure to convert back to Thermo temp and calculate error properly
    T_exc=convert_to_TT([(T_exc_N+T_exc_S)/2],file[1])[0]
    if type(file[4])==list:
        T_exc_err=np.sqrt((T_exc_N_err**2+T_exc_S_err**2)/4+file[4][2]**2)
    else:
        T_exc_err=np.sqrt((T_exc_N_err**2+T_exc_S_err**2)/4)

    print("T_exc at %sMHz is %.5f in the North, %.5f in the South w/ TT mean: %.5f pm %.5f" % (file[1],T_exc_N,T_exc_S,T_exc,T_exc_err))
    return T_exc,T_exc_err,alpha[0],alpha_err[0],alpha[1],alpha_err[1],alpha[2],alpha_err[2],file[1]


In [18]:
def ChiSquaredMinimizerHM(file,deg):
    #Chi^2 minimizer for Hongwan's Method-- WIP
    if file[1]==1420:#1420 MHz is the only one on this list (so far) that is in mK
        Map_as_array=(hp.read_map(file[0])/1000)

    else:
        Map_as_array=hp.read_map(file[0])
    
    nside=int(np.sqrt(len(Map_as_array)/12))
    FreeFree_Map=hp.ud_grade(hp.read_map("lambda_fds_dust_94GHz.fits"),nside).tolist()
    #Synch_Map=hp.ud_grade((hp.read_map("wmap_K_mem_synch_9yr_v5.fits")),nside).tolist()
    Synch_Map=hp.ud_grade(hp.ud_grade(hp.read_map("haslam408_dsds_Remazeilles2014.fits"),128)
                           -np.array(Rescaled408map)
                          ,nside).tolist()
    
    if file[3]==True:
        convert_to_TA(Map_as_array,file[1])    
    
    theta,phi=np.degrees(hp.pix2ang(nside,np.arange(len(Map_as_array))))
    if file[2]=="E":
        Map_as_array=MAP_Rotation_to_G(Map_as_array,"E")
    if file[2]=="C":
        Map_as_array=MAP_Rotation_to_G(Map_as_array,"C")
    Map_as_array=Map_as_array.tolist()
    

    if type(file[4])==str:
        sigma_N=[]
        sigma_S=[]
        sigmaMap=hp.read_map(file[4])
        if file[2]=="E":
            sigmaMap=MAP_Rotation_to_G(sigmaMap,"E")
        if file[2]=="C":
            sigmaMap=MAP_Rotation_to_G(sigmaMap,"C")
        sigmaMap=sigmaMap.tolist()
    
    theta_cut_N=[]
    Map_cut_N=[]
    Freefree_map_cut_N=[]
    Synch_map_cut_N=[]
    theta_cut_S=[]
    Map_cut_S=[]
    Freefree_map_cut_S=[]
    Synch_map_cut_S=[]
    
    for x in range(len(Map_as_array)):
        if Map_as_array[x]>0 and math.isnan(Map_as_array[x])==False and abs(90-theta[x])>deg and Synch_Map[x]>=0:
            if file[1]==2326:
                Map_as_array[x]+=2.505+0.325*(2326/2000)**(-2.75)
            if 90-theta[x]>0:
                if type(file[4])==list:            
                    Map_cut_N.append(Map_as_array[x])
                    Freefree_map_cut_N.append(FreeFree_Map[x])
                    Synch_map_cut_N.append(Synch_Map[x])
                    theta_cut_N.append(90-theta[x])
                if type(file[4])==str and sigmaMap[x]>0:
                    sigma_N.append(sigmaMap[x])
                    Map_cut_N.append(Map_as_array[x])
                    Freefree_map_cut_N.append(FreeFree_Map[x])
                    Synch_map_cut_N.append(Synch_Map[x])
                    theta_cut_N.append(90-theta[x])
            elif 90-theta[x]<0:
                if type(file[4])==list:            
                    Map_cut_S.append(Map_as_array[x])
                    Freefree_map_cut_S.append(FreeFree_Map[x])
                    Synch_map_cut_S.append(Synch_Map[x])
                    theta_cut_S.append(90-theta[x])
                if type(file[4])==str and sigmaMap[x]>0:
                    sigma_S.append(sigmaMap[x])
                    Map_cut_S.append(Map_as_array[x])
                    Freefree_map_cut_S.append(FreeFree_Map[x])
                    Synch_map_cut_S.append(Synch_Map[x])
                    theta_cut_S.append(90-theta[x])                    
    if type(file[4])==list:
        sigma_N=(np.sqrt(file[4][0]**2+(0.01*file[4][1]*np.array(Map_cut_N))**2)).tolist()
        sigma_S=(np.sqrt(file[4][0]**2+(0.01*file[4][1]*np.array(Map_cut_S))**2)).tolist()
        
        
    FreefreedumNorth=[]
    SynchdumNorth=[]
    FreefreedumSouth=[]
    SynchdumSouth=[]
    cscvals=[]
    for x in range(len(Map_as_array)):
        if 90-theta[x]>75:
            FreefreedumNorth.append(FreeFree_Map[x])
            SynchdumNorth.append(Synch_Map[x])
            cscvals.append(1/np.sin(np.radians(abs(90-theta[x]))))
        if 90-theta[x]<-75:
            FreefreedumSouth.append(FreeFree_Map[x])
            SynchdumSouth.append(Synch_Map[x])

    #alpha[3]=T_g
    
    ChiSquared = lambda alpha: np.sum((((np.array(Map_cut_N)-alpha[3]*np.mean(cscvals)-alpha[0]
                                         -alpha[1]*np.array(Freefree_map_cut_N-np.mean(FreefreedumNorth))
                                         -alpha[2]*np.array(Synch_map_cut_N-np.mean(SynchdumNorth)))/(sigma_N))**2))+ np.sum(((
        np.array(Map_cut_N)-alpha[3]/np.sin(np.radians(abs(np.array(theta_cut_N))))-alpha[0])/(sigma_N))**2)+np.sum((((
        np.array(Map_cut_S)-alpha[4]*np.mean(cscvals)-alpha[5]
                                         -alpha[1]*np.array(Freefree_map_cut_S-np.mean(FreefreedumSouth))
                                         -alpha[2]*np.array(Synch_map_cut_S-np.mean(SynchdumSouth)))/(sigma_S))**2))+ np.sum(((
        np.array(Map_cut_S)-alpha[4]/np.sin(np.radians(abs(np.array(theta_cut_S))))-alpha[5])/(sigma_S))**2)
    bnds = ((-np.inf, np.inf),(0, np.inf),(0, np.inf),(-np.inf,np.inf),(-np.inf, np.inf),(-np.inf, np.inf))
    maps=minimize(ChiSquared,x0=(np.mean(Map_cut_S),0.1,1,np.mean(Map_cut_S),np.mean(Map_cut_S),np.mean(Map_cut_S)),bounds=bnds)
    alpha=maps.x
    T_g_N=alpha[3]
    T_exc_N=alpha[0]
    T_g_S=alpha[4]
    T_exc_S=alpha[5]
    T_exc=convert_to_TT([(T_exc_N+T_exc_S)/2],file[1])[0]
    T_g=convert_to_TT([(T_g_N+T_g_S)/2],file[1])[0]
    print("T_exc at %sMHz is %.5f in the North, %.5f in the South w/ TT mean: %.5f" % (file[1],T_exc_N,T_exc_S,T_exc))
    print("T_G at %sMHz is %.5f in the North, %.5f in the South w/ TT mean: %.5f" % (file[1],T_g_N,T_g_S,T_g))
    print("")
    return(alpha[0],alpha[1],alpha[2],T_exc)